In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf

import math
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import gzip
import tensorflow as tf
# tf.random.set_seed(1004)
tf.random.set_seed(2)

# Data Load

In [5]:
label = np.load("../data/2030days_label.npy")
data_10days = np.load("../data/10days_data.npy")
label.shape, data_10days.shape

((7799,), (7799, 10, 4069))

In [6]:
def MAE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.abs(y_true-y_pred)))

def MSE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.square(y_true-y_pred)))

def RMSE(y_true, y_pred):
    return "{:.2e}".format(np.sqrt(np.mean(np.square(y_true-y_pred))))

In [7]:
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from keras.layers import InputLayer, GRU

In [8]:
# 사용할 data 지정
data = data_10days

#train, test split
i = round(data.shape[0] * 0.8)
X_train = data[:i,:,:]
y_train = label[:i]


X_test = data[i:,:,:]
y_test = label[i:]


X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6239, 10, 4069), (6239,), (1560, 10, 4069), (1560,))

# Modeling

- seed(1004) : 73.3333333
- seed(5555) : 73.1

- tf.random.set_seed(1) : 정확도 : 0.7403846153846154
- tf.random.set_seed(10) : 

In [9]:
%%time
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dense(units=1))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           2149376   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________
[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f482be48f70>), ('model2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f482bb9ef10>), ('model3', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f48282

25/25 [==============================] - 2s 75ms/step - loss: 0.1055 - acc: 0.9726
Epoch 14/20
25/25 [==============================] - 2s 75ms/step - loss: 0.1021 - acc: 0.9732
Epoch 15/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0995 - acc: 0.9748
Epoch 16/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0970 - acc: 0.9747
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0984 - acc: 0.9756
Epoch 18/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0905 - acc: 0.9771
Epoch 19/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0881 - acc: 0.9774
Epoch 20/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0860 - acc: 0.9787
Epoch 1/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0911 - acc: 0.9772
Epoch 2/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0853 - acc: 0.9790
Epoch 3/20
25/25 [==============================] - 2s 74ms/step

25/25 [==============================] - 2s 74ms/step - loss: 0.0439 - acc: 0.9939
Epoch 20/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0439 - acc: 0.9936
Epoch 1/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0442 - acc: 0.9929
Epoch 2/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0434 - acc: 0.9936
Epoch 3/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0438 - acc: 0.9942
Epoch 4/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0415 - acc: 0.9936
Epoch 5/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0429 - acc: 0.9937
Epoch 6/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0429 - acc: 0.9941
Epoch 7/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0432 - acc: 0.9937
Epoch 8/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0433 - acc: 0.9949
Epoch 9/20
25/25 [==============================] - 2s 75ms/step - los

25/25 [==============================] - 2s 73ms/step - loss: 0.0311 - acc: 0.9960
Epoch 6/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0311 - acc: 0.9962
Epoch 7/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0311 - acc: 0.9957
Epoch 8/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0315 - acc: 0.9958
Epoch 9/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0312 - acc: 0.9962
Epoch 10/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0311 - acc: 0.9960
Epoch 11/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0308 - acc: 0.9960
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0311 - acc: 0.9957
Epoch 13/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0309 - acc: 0.9962
Epoch 14/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0308 - acc: 0.9960
Epoch 15/20
25/25 [==============================] - 2s 74ms/step 

25/25 [==============================] - 2s 74ms/step - loss: 0.0470 - acc: 0.9952
Epoch 12/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0470 - acc: 0.9950
Epoch 13/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0471 - acc: 0.9952
Epoch 14/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0469 - acc: 0.9952
Epoch 15/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0468 - acc: 0.9952
Epoch 16/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0471 - acc: 0.9952
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0479 - acc: 0.9952
Epoch 18/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0472 - acc: 0.9952
Epoch 19/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0469 - acc: 0.9952
Epoch 20/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0469 - acc: 0.9952
Epoch 1/20
25/25 [==============================] - 2s 73ms/st

...

25/25 [==============================] - 2s 75ms/step - loss: 0.0296 - acc: 0.9965
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0294 - acc: 0.9965
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0299 - acc: 0.9965
Epoch 18/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0296 - acc: 0.9965
Epoch 19/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0295 - acc: 0.9965
Epoch 20/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0294 - acc: 0.9965
Epoch 1/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0295 - acc: 0.9965
Epoch 2/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0294 - acc: 0.9965
Epoch 3/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0295 - acc: 0.9965
Epoch 4/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0294 - acc: 0.9965
Epoch 5/20
25/25 [==============================] - 2s 72ms/step -

25/25 [==============================] - 2s 73ms/step - loss: 0.0270 - acc: 0.9966
Epoch 2/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0269 - acc: 0.9966
Epoch 3/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0271 - acc: 0.9966
Epoch 4/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0269 - acc: 0.9966
Epoch 5/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0270 - acc: 0.9966
Epoch 6/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0270 - acc: 0.9966
Epoch 7/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0272 - acc: 0.9966
Epoch 8/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0271 - acc: 0.9966
Epoch 9/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0272 - acc: 0.9966
Epoch 10/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0271 - acc: 0.9966
Epoch 11/20
25/25 [==============================] - 2s 74ms/step - lo

25/25 [==============================] - 2s 74ms/step - loss: 0.0222 - acc: 0.9970
Epoch 8/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0221 - acc: 0.9970
Epoch 9/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0222 - acc: 0.9970
Epoch 10/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0221 - acc: 0.9970
Epoch 11/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0221 - acc: 0.9970
Epoch 12/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0222 - acc: 0.9970
Epoch 13/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0220 - acc: 0.9970
Epoch 14/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0220 - acc: 0.9970
Epoch 15/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0221 - acc: 0.9970
Epoch 16/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0220 - acc: 0.9970
Epoch 17/20
25/25 [==============================] - 2s 73ms/ste

25/25 [==============================] - 2s 74ms/step - loss: 0.0222 - acc: 0.9970
Epoch 14/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0221 - acc: 0.9970
Epoch 15/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0221 - acc: 0.9970
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0220 - acc: 0.9970
Epoch 17/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0223 - acc: 0.9970
Epoch 18/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0221 - acc: 0.9970
Epoch 19/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0220 - acc: 0.9970
Epoch 20/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0220 - acc: 0.9970
Epoch 1/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0220 - acc: 0.9970
Epoch 2/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0220 - acc: 0.9970
Epoch 3/20
25/25 [==============================] - 2s 72ms/step

25/25 [==============================] - 2s 73ms/step - loss: 0.0220 - acc: 0.9970
Epoch 20/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0220 - acc: 0.9970
Epoch 1/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0220 - acc: 0.9970
Epoch 2/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0220 - acc: 0.9970
Epoch 3/20
25/25 [==============================] - 2s 71ms/step - loss: 0.0221 - acc: 0.9970
Epoch 4/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0220 - acc: 0.9970
Epoch 5/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0220 - acc: 0.9970
Epoch 6/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0220 - acc: 0.9970
Epoch 7/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0222 - acc: 0.9970
Epoch 8/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0220 - acc: 0.9970
Epoch 9/20
25/25 [==============================] - 2s 75ms/step - los